In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests

In [1]:
from bs4 import BeautifulSoup
import requests
import logging
import http.client
http.client.HTTPConnection.debuglevel = 1
import pandas as pd
import time
import re
import os

In [2]:
# List of Hotel's URL
list_hotel = pd.read_excel('fix_hotel.xlsx', sheet_name='Sheet4')
list_hotel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   hotel         50 non-null     object
 1   hotel_link    50 non-null     object
 2   review_count  50 non-null     int64 
 3   hotel_name    50 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.7+ KB


In [3]:
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)
requests_log = logging.getLogger("requests.packages.urllib3")
requests_log.setLevel(logging.DEBUG)
requests_log.propagate = True

In [4]:
def hotelScraper():
    for hotel in list_hotel.index:
        hotel_name = list_hotel['hotel'][hotel]
        hotel_url = list_hotel['hotel_link'][hotel]
        filename = list_hotel['hotel_name'][hotel]
        filepath = 'Dataset_1/' + filename + '.csv'

        if os.path.exists(filepath):
            continue
        
        
        print(f"Start Scrapping: {hotel_name}")
        
        gathered_review = []
        headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
        pages = 200
        pages = (pages * 5) + 5
        for i in range(5, pages, 5):
            new_url = re.sub('or5', 'or'+str(i), str(hotel_url))
            url = requests.get(new_url, headers=headers)

            soup = BeautifulSoup(url.text, 'lxml')
            reviews = soup.find_all('div', class_= 'YibKl MC R2 Gi z Z BB pBbQr')
            
            if reviews:
                for review in reviews:
                    user_text = review.find('span', class_= 'QewHA H4 _a').text
                    user_rating = review.find('span', class_= 'ui_bubble_rating')['class'][1][7]
                    gathered_review.append([hotel_name, user_text, user_rating])

                #Pages Processed
                cpages = int(i/5)
                print(f"Done Scrapping {hotel_name} page {cpages}")
                print('\n')
                time.sleep(3)

                # Make csv file
                df = pd.DataFrame(gathered_review, columns=['hotel', 'review', 'stars'])
                df.to_csv('Dataset_1/'+filename+'.csv')
            else:
                if os.path.exists(filepath):
                    break
                else:
                    hotelScraper()
    
        

In [6]:
hotelScraper()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.tripadvisor.co.id:443


Start Scrapping: Le Grande Bali
send: b'GET /Hotel_Review-g1380108-d1379463-Reviews-or5-Le_Grande_Bali-Pecatu_Bukit_Peninsula_Bali.html HTTP/1.1\r\nHost: www.tripadvisor.co.id\r\nUser-Agent: Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'


DEBUG:urllib3.connectionpool:https://www.tripadvisor.co.id:443 "GET /Hotel_Review-g1380108-d1379463-Reviews-or5-Le_Grande_Bali-Pecatu_Bukit_Peninsula_Bali.html HTTP/1.1" 200 None


reply: 'HTTP/1.1 200 OK\r\n'
header: Server: envoy
header: Link: <https://media-cdn.tripadvisor.com/media/>;rel="preconnect";crossorigin
header: Link: <https://media-cdn.tripadvisor.com/media/>;rel="dns-prefetch"
header: p3p: CP="NOI CURa ADMa DEVa TAIa OUR BUS IND UNI COM NAV INT DSP COR CAO PSA IDC ADM DEVi TAIi PSD IVAi IVDi CONi HIS CNT"
header: Pragma: no-cache
header: Cache-Control: no-cache, no-store, must-revalidate
header: Expires: 0
header: Content-Language: id
header: Link: <https://static.tacdn.com>;rel="dns-prefetch"
header: Link: <https://static.tacdn.com>;rel="preconnect";crossorigin
header: timing-allow-origin: https://www.tripadvisor.com
header: Vary: Accept-Encoding
header: Content-Type: text/html;charset=UTF-8
header: x-datadome: protected
header: Accept-CH: Sec-CH-UA,Sec-CH-UA-Mobile,Sec-CH-UA-Platform,Sec-CH-UA-Arch,Sec-CH-UA-Full-Version-List,Sec-CH-UA-Model,Sec-CH-Device-Memory
header: Content-Encoding: gzip
header: x-request-id: 61a00e0d-d988-4273-9791-9f539e27d

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.tripadvisor.co.id:443


...

reply: 'HTTP/1.1 200 OK\r\n'
header: Server: envoy
header: Link: <https://media-cdn.tripadvisor.com/media/>;rel="preconnect";crossorigin
header: Link: <https://media-cdn.tripadvisor.com/media/>;rel="dns-prefetch"
header: p3p: CP="NOI CURa ADMa DEVa TAIa OUR BUS IND UNI COM NAV INT DSP COR CAO PSA IDC ADM DEVi TAIi PSD IVAi IVDi CONi HIS CNT"
header: Pragma: no-cache
header: Cache-Control: no-cache, no-store, must-revalidate
header: Expires: 0
header: Content-Language: id
header: Link: <https://static.tacdn.com>;rel="dns-prefetch"
header: Link: <https://static.tacdn.com>;rel="preconnect";crossorigin
header: timing-allow-origin: https://www.tripadvisor.com
header: Vary: Accept-Encoding
header: Content-Type: text/html;charset=UTF-8
header: x-datadome: protected
header: Accept-CH: Sec-CH-UA,Sec-CH-UA-Mobile,Sec-CH-UA-Platform,Sec-CH-UA-Arch,Sec-CH-UA-Full-Version-List,Sec-CH-UA-Model,Sec-CH-Device-Memory
header: Content-Encoding: gzip
header: x-request-id: 13eb44b5-37e5-4857-9cf9-769b60f76

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.tripadvisor.co.id:443


send: b'GET /Hotel_Review-g297701-d479525-Reviews-or295-Pertiwi_Resort_Spa-Ubud_Gianyar_Regency_Bali.html HTTP/1.1\r\nHost: www.tripadvisor.co.id\r\nUser-Agent: Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'


DEBUG:urllib3.connectionpool:https://www.tripadvisor.co.id:443 "GET /Hotel_Review-g297701-d479525-Reviews-or295-Pertiwi_Resort_Spa-Ubud_Gianyar_Regency_Bali.html HTTP/1.1" 200 None


reply: 'HTTP/1.1 200 OK\r\n'
header: Server: envoy
header: Link: <https://media-cdn.tripadvisor.com/media/>;rel="preconnect";crossorigin
header: Link: <https://media-cdn.tripadvisor.com/media/>;rel="dns-prefetch"
header: p3p: CP="NOI CURa ADMa DEVa TAIa OUR BUS IND UNI COM NAV INT DSP COR CAO PSA IDC ADM DEVi TAIi PSD IVAi IVDi CONi HIS CNT"
header: Pragma: no-cache
header: Cache-Control: no-cache, no-store, must-revalidate
header: Expires: 0
header: Content-Language: id
header: Link: <https://static.tacdn.com>;rel="dns-prefetch"
header: Link: <https://static.tacdn.com>;rel="preconnect";crossorigin
header: timing-allow-origin: https://www.tripadvisor.com
header: Vary: Accept-Encoding
header: Content-Type: text/html;charset=UTF-8
header: x-datadome: protected
header: Accept-CH: Sec-CH-UA,Sec-CH-UA-Mobile,Sec-CH-UA-Platform,Sec-CH-UA-Arch,Sec-CH-UA-Full-Version-List,Sec-CH-UA-Model,Sec-CH-Device-Memory
header: Content-Encoding: gzip
header: x-request-id: 7bb807c4-2c02-4699-8885-d727c454e